In [1]:
storage_account_name = "<Storage account name>"
storage_account_key = "<Storage account key>"
container = "<Container name>"

spark.conf.set("fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account_name), storage_account_key)

In [2]:
%fs ls wasbs://inbound@databricksdemostorage.blob.core.windows.net/

In [3]:
dbutils.fs.ls("wasbs://inbound@databricksdemostorage.blob.core.windows.net/")

In [4]:
df = spark.read \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("delimiter", ";") \
  .csv("wasbs://{0}@{1}.blob.core.windows.net/<File name>".format(container, storage_account_name))

df.show()

In [5]:
dbutils.fs.mount(
  source = "wasbs://{0}@{1}.blob.core.windows.net".format(container, storage_account_name),
  mount_point = "/mnt/azureStorage",
  extra_configs = {"fs.azure.account.key.{0}.blob.core.windows.net".format(storage_account_name): storage_account_key}
)

In [6]:
%fs ls "dbfs:/mnt/azureStorage"

In [7]:
df2 = spark.read \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("delimiter", ";") \
  .csv("/mnt/azureStorage/<File name>")

df2.show()

In [8]:
data = spark.createDataFrame([(22, "John")], ["Age", "FirstName"])

In [9]:
data.write.csv("wasbs://{0}@{1}.blob.core.windows.net/new.csv".format(container, storage_account_name))

In [10]:
dbutils.fs.ls("wasbs://{0}@{1}.blob.core.windows.net/".format(container, storage_account_name))

In [11]:
data.coalesce(1) \
  .write \
  .option("header", "true") \
  .option("delimiter", ",") \
  .csv("wasbs://<Container name>@<Account name>.blob.core.windows.net/<File name>")

In [12]:
dbutils.library.installPyPI("azure.storage.blob")

In [13]:
pandas_df = data.toPandas()
pandas_df.to_csv("/dbfs/FileStore/tables/<File name>", encoding = "utf-8")

In [14]:
file = dbutils.fs.head("dbfs:/FileStore/tables/<File name>")

In [15]:
from azure.storage.blob import BlockBlobService

blobService = BlockBlobService(storage_account_name, storage_account_key)

blobService.create_blob_from_text('<Container name>', '<File name>', file)